In [5]:
import cv2
import numpy as np

# Coin classification
def coin_classification(radius, mean_color_bgr):
    b, g, r = mean_color_bgr

    # Silver coins
    is_silver = abs(r - g) < 20 and abs(g - b) < 20 and abs(r - b) < 20

    # Gold coins
    is_gold = (r > g + 15) and (r > b + 15)

    if 25 <= radius < 33 and is_silver:
        return 'PHP 1', 1
    elif 33 <= radius < 40 and is_gold:
        return 'PHP 5', 5
    elif 40 <= radius < 55:
        return 'PHP 10', 10
    else:
        return None, 0

# Start webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=50,
        param1=100,
        param2=40,
        minRadius=15,
        maxRadius=60
    )

    output = frame.copy()
    coin_count, total_value = 0, 0.0

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            
            mask = np.zeros(frame.shape[:2], dtype="uint8")
            cv2.circle(mask, (x, y), r, 255, -1)

            mean_color = cv2.mean(frame, mask=mask)[:3]

            label, value = coin_classification(r, mean_color)

            if value > 0:
                coin_count += 1
                total_value += value
                cv2.circle(output, (x, y), r, (0, 255, 0), 2)
                cv2.putText(output, f"{label}", (x - 40, y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Overlay totals
    cv2.putText(output, f"Total coins: {coin_count}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(output, f"Total value: PHP {total_value:.2f}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

    cv2.imshow("Coin Detector", output)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
